# Getting Acquainted with SQL

Now that we are familiar with data carpentry and data types, we should familiarize ourselves with databases and how to interact with them. We will explore the basic concepts of SQL (structured query language) and database configuration and by the end of the module we will understand the power and portability of databases.

The purpose of this notebook is to get our feet wet with SQL. We are particularly interested in the components that make up a query. Below, you will be exposed to data loading, but this portion will be deemphasized in order for us to get into the basics of constructing an SQL query. Don't worry, however, the next notebook will get into the some weeds of loading data into a database. With all of this being just a sampling of what is to come in the Database course. 

But why? Why use a database? Some technical reasons include:
1. You can pull data out of them fairly simply
2. They have structure and can store multiple tables 
3. Tables can easily be related to one another 
4. They can store large amounts of data
5. They are concurrent; they allow for multiple users to access them at the same time
6. They are portable and scalable

So far in this course we have only be exposed to `Python` and `R` so is it really necessary to learn `SQL`?
The simple answer to this is yes. `SQL` is the language of databases and a lot (most) of companies store their data in databases. 

There are several different flavors of databases. 
You may have heard of some: mysql, postgresql, Oracle, Microsoft SQL Server. 
In this notebook we will be working with SQLite. 
As its name suggest, SQLite doesn't have all of the features as the other systems, but it does adhere to the SQL language and is easy to get up and running. 
For that reason, we are starting here.

Let's dive in...

In [ ]:
import pandas as pd
import sqlite3
import os 

In [ ]:
actor = pd.DataFrame({
    'name': ['Natalie Portman','Bill Murray', 'Will Smith', 'Kathy Bates'],
    'age' : [35,66, 48, 69],
    'birthplace' : ['Jerusalem, Israel', 'Evanston, Illinois', 'Philadelphia, Pennsylvania', 'Memphis, Tennessee']
             })

salary = pd.DataFrame({
    'name': ['Natalie Portman', 'Natalie Portman', 'Bill Murray', 'Bill Murray','Will Smith','Will Smith','Kathy Bates', 'Kathy Bates', 'Viola Davis'],
    'year': [2000, 2017, 2000, 2017, 2000, 2017, 2000, 2017, 2017 ],
    'salary': [500000, 10000000, 1000000, 5000000, 750000, 10000000, 1200000,9000000, 8000000]
})

Nothing different or new here. The one thing to keep in mind is that we are working with dummy data. The validity of these data points cannot be verified; the salary column is a straight up fabrication. You may realize either from context clues or knowledge of pop culture that we are dealing with actors and actresses. Whether or not you are familiar with these four is irrelevant. 

We just created two data frames. Why two? Well, we can reduce data redundancies when we split these up into two different tables. In other words, the attributes of the `actor` data frame would have to be repeated twice if joined with the `salary` table since it keeps record of an actor's salary for both the year 2000 and 2017. This concept is called `normalization` and will be covered more in the database course.

In [ ]:
print(actor)
print("---------")
print(salary)

And below is where we create a database called `sample.db`. It exists in the directory above the directory we are currently in, meaning in the module sub directory and not the labs sub directory. 

Now, I don't want to get into the thick of what is going on here, primarily because we will cover this in the next lab. Just know that we are using `pandas` to create tables and then populate those tables with the data. 

In [ ]:
db_file_name = '../sample.db'

if os.path.exists(db_file_name):
    os.remove(db_file_name)
    
# Open / Create the sample.db database file.
connection = sqlite3.connect(db_file_name)

# Note: Each command below inspects the data and creates a table, 
# then populates the data into the table
salary.to_sql('salary', connection, if_exists='replace')
actor.to_sql('actor', connection, if_exists='replace')

### Querying the Database

Our data is now in a SQLite database...now what? Access it! 

Now keep in mind that this is just one way to access data from a database but you can connect to a database through many languages including `R`. Below is just a simple way to open a database connection and run a query and return the results to a `pandas` data frame. The important part below is the query (the stuff in quotes). 

#### `SELECT`

The most basic component of querying in SQL is the `select` statement. You use this statement to tell the database which attributes (columns) you are interested in. After you specify the attributes, you tell SQL which table these attributes are from. Imagine that I wanted to see all of the values from the all of the attributes in the salary table. You would write

```SQL
SELECT *
FROM salary;
```

The `*` tells SQL that you want all the attributes. Take a look for yourself by running the code below.

In [ ]:
pd.read_sql_query("SELECT * FROM salary;", connection)

So we understand what is going on, `pandas` has a function called `read_sql_query` that takes in a string containing a SQL query as the first argument and the database connection as the second. We open this connection two code cells above with this line:

```python
connection = sqlite3.connect(db_file_name)
```

Where `db_file_name` is the database file. This function then returns the results to a `pandas` dataframe.

We can also specify a single column to pull like so

```SQL
SELECT <column_name>
FROM <table_name>;
```

In [ ]:
query = "SELECT name FROM salary;"
pd.read_sql_query(query, connection)

...or multiple columns

```SQL
SELECT <column_name> , <column_name>
FROM <table_name>;
```

In [ ]:
query = "SELECT name, year FROM salary;"
pd.read_sql_query(query, connection)

#### `WHERE`

The `select` statement allows us to subset a table columnwise, but what if we want a rowwise subset? The `where` clause achieves that. The `where` statement allows us to put a condition on the attributes so that we only return rows that meet that condition.

```SQL
SELECT <column_names>
FROM <table_name>
WHERE <column_name> [=<>!] <condition>;
```

For instances, if we only wanted rows from the salary table where the `year` is 2017:

In [ ]:
query = "SELECT * FROM salary WHERE year = 2017;"
pd.read_sql_query(query, connection)

#### `ORDER BY`

Sometimes you may just want to see what the data looks like from an ordered perspective. For example, a very reasonable thing to want to know is who had what salary from least to greatest during the year of 2017. 

```SQL
SELECT <column_name>
FROM <table_name>
ORDER BY <column_name>;
```

*Again, remember these data are made up for the purpose of demonstration. I have no idea what these actors make although they definitely made more than yours truly did in my acting career *

In [ ]:
query = "SELECT * FROM salary WHERE year = 2017 ORDER BY salary;"
pd.read_sql_query(query, connection)

...and perhaps you wanted to know the order from greatest to least. We didn't need to specify an order for the least to greatest as the default order is ascending. However, we could have by placing `ASC` after the column. As you can see below for descending order (greatest to least) we use `DESC`.

In [ ]:
query = "SELECT * FROM salary WHERE year = 2017 ORDER BY salary DESC;"
pd.read_sql_query(query, connection)

**What do you think will happen if you order a query by a non-numeric attribute?**

In [ ]:
# Play around with non-numericattribute ordering here
# ------------------

query = "SELECT name FROM salary WHERE year = 2017 ORDER BY name;"
pd.read_sql_query(query, connection)

### `JOIN`s

Joins are among SQL's most powerful features. Joins allow us to relate one table to another, returning the results to us as a single table. Now, `SQLite` doesn't include all the different types of `joins` but it certainly has enough to display the concept's benefits. 

#### `CROSS JOIN`

`CROSS JOIN`ing one table to another joins every row in table A to every row in table B. **A WARNING ABOUT `CROSS JOIN`S**: they have the tendency to get very large so use with caution. For our dummy case today we should only expect a table of 36 rows X 8 columns ( (num rows in A X num rows in B) **BY** (num cols in A + num cols in B) ).

![CrossJoin](../images/crossJoin.jpeg)

In [ ]:
query = "SELECT * FROM actor CROSS JOIN salary"
pd.read_sql_query(query, connection)

#### `INNER JOIN`

`INNER JOIN`s are going to return only those rows that have a match in both tables. We can specify which column we want to join by in `SQLite` by using the `USING` statement. In the case below we are going to join by `name` as this is the only feature that exists in both tables. We can expect that all rows that have a matching `name` in both tables will be returned, therefore, all actors save for Viola Davis will be returned to us. Viola Davis doesn't have a row in the `actor` table.


![ven-inner-join](../images/sql-inner-join.png)

![inner join](../images/innerJoin.jpeg)

In [ ]:
query = "SELECT * FROM actor INNER JOIN salary USING (name);"
pd.read_sql_query(query, connection)

And we can reverse the table order for this query to see what behavior occurs. You will notice that the table mentioned first will have its attributes displayed on the left...

In [ ]:
query = "SELECT * FROM salary INNER JOIN actor USING (name);"
pd.read_sql_query(query, connection)

#### `OUTER JOINS`

There are several different types of `OUTER JOIN`s but `SQLite` only supports one -  `LEFT OUTER JOIN`. `OUTER JOINS` are going to match on a specific column but will return Null values if there isn't a match. A `LEFT OUTER JOIN` will ensure that all of the columns on the left side table (the table listed first in the query) are present regardless of whether or not there is an analogous value in the matching attribute column. 

Again, let's specify `name` as our matching column. Instead of using `USING`, you can also use `ON` and specify the table and column name that match using the `tablename`.`attributename` syntax seen below.

![VenouterJoin](../images/sql-left-join.png)

In [ ]:
query = "SELECT * FROM actor LEFT OUTER JOIN salary ON actor.name = salary.name;"
pd.read_sql_query(query, connection)

This should look familiar. In fact, it is the same result that we got for the first `INNER JOIN` that we committed above. That is because the `salary` table contains all of the names that are found in the `actor` table. However, what if the `salary` table is on the left?

![Left Outer Join](../images/LeftOuterJoin.jpeg)

In [ ]:
query = "SELECT * FROM salary LEFT OUTER JOIN actor ON actor.name = salary.name;"
pd.read_sql_query(query, connection)

There we go. You might have guessed it. Since Viola Davis doesn't have any rows in the `actor` table, the values under the attributes brought over from the `actor` table have `NaN` values assigned to them.

# Save your notebook, then `File > Close and Halt`